# Heat Equation

![](heat_square.png)

*Figure 1*: Temperature field on the unit square with an internal uniform heat source
solved with homogeneous Dirichlet boundary conditions on the boundary.

## Introduction

The heat equation is the "Hello, world!" equation of finite elements.
Here we solve the equation on a unit square, with a uniform internal source.
The strong form of the (linear) heat equation is given by

$$
 -\nabla \cdot (k \nabla u) = f  \quad \textbf{x} \in \Omega,
$$

where $u$ is the unknown temperature field, $k$ the heat conductivity,
$f$ the heat source and $\Omega$ the domain. For simplicity we set $f = 1$
and $k = 1$. We will consider homogeneous Dirichlet boundary conditions such that
$$
u(\textbf{x}) = 0 \quad \textbf{x} \in \partial \Omega,
$$
where $\partial \Omega$ denotes the boundary of $\Omega$.
The resulting weak form is given given as follows: Find $u \in \mathbb{U}$ such that
$$
\int_{\Omega} \nabla \delta u \cdot \nabla u \ d\Omega = \int_{\Omega} \delta u \ d\Omega \quad \forall \delta u \in \mathbb{T},
$$
where $\delta u$ is a test function, and where $\mathbb{U}$ and $\mathbb{T}$ are suitable
trial and test function sets, respectively.

## Commented Program

Now we solve the problem in Ferrite. What follows is a program spliced with comments.

First we load Ferrite, and some other packages we need

In [1]:
using Ferrite, SparseArrays

We start by generating a simple grid with 20x20 quadrilateral elements
using `generate_grid`. The generator defaults to the unit square,
so we don't need to specify the corners of the domain.

In [2]:
grid = generate_grid(Quadrilateral, (20, 20));

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the two-dimensional reference "cube". We also define a quadrature rule based on
the same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

In [20]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip);

Lagrange{2, RefCube, 1}()


### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single scalar field called `:u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

In [4]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct entries stored.

In [5]:
K = create_sparsity_pattern(dh)

441×441 SparseMatrixCSC{Float64, Int64} with 3721 stored entries:
⎡⠻⣦⡀⠀⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⠈⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠉⠓⠦⣄⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⢎⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⣮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡱⣮⡳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⡳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡻⣮⠳⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡻⣮⡳⣄⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⠻⣦⎦

### Boundary conditions
In Ferrite constraints like Dirichlet boundary conditions
are handled by a `ConstraintHandler`.

In [6]:
ch = ConstraintHandler(dh);

Next we need to add constraints to `ch`. For this problem we define
homogeneous Dirichlet boundary conditions on the whole boundary, i.e.
the `union` of all the face sets on the boundary.

In [7]:
∂Ω = union(
    getfaceset(grid, "left"),
    getfaceset(grid, "right"),
    getfaceset(grid, "top"),
    getfaceset(grid, "bottom"),
);

Now we are set up to define our constraint. We specify which field
the condition is for, and our combined face set `∂Ω`. The last
argument is a function of the form $f(\textbf{x})$ or $f(\textbf{x}, t)$,
where $\textbf{x}$ is the spatial coordinate and
$t$ the current time, and returns the prescribed value. Since the boundary condition in
this case do not depend on time we define our function as $f(\textbf{x}) = 0$, i.e.
no matter what $\textbf{x}$ we return $0$. When we have
specified our constraint we `add!` it to `ch`.

In [8]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> 0)
add!(ch, dbc);

Finally we also need to `close!` our constraint handler. When we call `close!`
the dofs corresponding to our constraints are calculated and stored
in our `ch` object.

In [9]:
close!(ch)

ConstraintHandler:
  BCs:
    Field: u, Components: [1]

Note that if one or more of the constraints are time dependent we would use
`update!` to recompute prescribed values in each new timestep.

### Assembling the linear system

Now we have all the pieces needed to assemble the linear system, $K u = f$.
Assembling of the global system is done by looping over all the elements in order to
compute the element contributions $K_e$ and $f_e$, which are then assembled to the
appropriate place in the global $K$ and $f$.

#### Element assembly
We define the function `assemble_element!` (see below) which computes the contribution for
an element. The function takes pre-allocated `ke` and `fe` (they are allocated once and
then reused for all elements) so we first need to make sure that they are all zeroes at
the start of the function by using `fill!`. Then we loop over all the quadrature points,
and for each quadrature point we loop over all the (local) shape functions. We need the
value and gradient of the test function, `δu` and also the gradient of the trial function
`u`. We get all of these from `cellvalues`.

!!! note "Notation"
    Comparing with the brief finite element introduction in Introduction to FEM,
    the variables `δu`, `∇δu` and `∇u` are actually $\phi_i(\textbf{x}_q)$, $\nabla
    \phi_i(\textbf{x}_q)$ and $\nabla \phi_j(\textbf{x}_q)$, i.e. the evaluation of the
    trial and test functions in the quadrature point $\textbf{x}_q$. However, to
    underline the strong parallel between the weak form and the implementation, this
    example uses the symbols appearing in the weak form.

In [21]:
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        println("dΩ = ", dΩ)
        println("q_point = ", q_point)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # Add contribution to Ke
                Ke[i, j] += (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

assemble_element! (generic function with 1 method)

#### Global assembly
We define the function `assemble_global` to loop over the elements and do the global
assembly. The function takes our `cellvalues`, the sparse matrix `K`, and our DofHandler
as input arguments and returns the assembled global stiffness matrix, and the assembled
global force vector. We start by allocating `Ke`, `fe`, and the global force vector `f`.
We also create an assembler by using `start_assemble`. The assembler lets us assemble into
`K` and `f` efficiently. We then start the loop over all the elements. In each loop
iteration we reinitialize `cellvalues` (to update derivatives of shape functions etc.),
compute the element contribution with `assemble_element!`, and then assemble into the
global `K` and `f` with `assemble!`.

!!! note "Notation"
    Comparing again with Introduction to FEM, `f` and `u` correspond to
    $\underline{\hat{f}}$ and $\underline{\hat{u}}$, since they represent the discretized
    versions. However, through the code we use `f` and `u` instead to reflect the strong
    connection between the weak form and the Ferrite implementation.

In [22]:
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Compute element contribution
        assemble_element!(Ke, fe, cellvalues)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

### Solution of the system
The last step is to solve the system. First we call `assemble_global`
to obtain the global stiffness matrix `K` and force vector `f`.

In [23]:
K, f = assemble_global(cellvalues, K, dh);

dΩ = 0.002500000000000004
q_point = 1
dΩ = 0.0025000000000000044
q_point = 2
dΩ = 0.002500000000000003
q_point = 3
dΩ = 0.002500000000000004
q_point = 4
dΩ = 0.0024999999999999974
q_point = 1
dΩ = 0.0024999999999999983
q_point = 2
dΩ = 0.0024999999999999974
q_point = 3
dΩ = 0.0024999999999999983
q_point = 4
dΩ = 0.002500000000000002
q_point = 1
dΩ = 0.0025000000000000027
q_point = 2
dΩ = 0.002500000000000003
q_point = 3
dΩ = 0.002500000000000004
q_point = 4
dΩ = 0.002499999999999999
q_point = 1
dΩ = 0.0024999999999999996
q_point = 2
dΩ = 0.002499999999999999
q_point = 3
dΩ = 0.0024999999999999996
q_point = 4
dΩ = 0.0025000000000000027
q_point = 1
dΩ = 0.002500000000000003
q_point = 2
dΩ = 0.002500000000000002
q_point = 3
dΩ = 0.0025000000000000027
q_point = 4
dΩ = 0.002500000000000001
q_point = 1
dΩ = 0.002500000000000002
q_point = 2
dΩ = 0.0025000000000000005
q_point = 3
dΩ = 0.002500000000000001
q_point = 4
dΩ = 0.0025
q_point = 1
dΩ = 0.002500000000000001
q_point = 2
dΩ = 0.00250000

To account for the boundary conditions we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [16]:
apply!(K, f, ch)
u = K \ f;

DofHandler{2, Grid{2, Quadrilateral, Float64}}([:u], [1], Interpolation[Lagrange{2, RefCube, 1}()], [1, 2, 3, 4, 2, 5, 6, 3, 5, 7, 8, 6, 7, 9, 10, 8, 9, 11, 12, 10, 11, 13, 14, 12, 13, 15, 16, 14, 15, 17, 18, 16, 17, 19, 20, 18, 19, 21, 22, 20, 21, 23, 24, 22, 23, 25, 26, 24, 25, 27, 28, 26, 27, 29, 30, 28, 29, 31, 32, 30, 31, 33, 34, 32, 33, 35, 36, 34, 35, 37, 38, 36, 37, 39, 40, 38, 39, 41, 42, 40, 4, 3, 43, 44, 3, 6, 45, 43, 6, 8, 46, 45, 8, 10, 47, 46, 10, 12, 48, 47, 12, 14, 49, 48, 14, 16, 50, 49, 16, 18, 51, 50, 18, 20, 52, 51, 20, 22, 53, 52, 22, 24, 54, 53, 24, 26, 55, 54, 26, 28, 56, 55, 28, 30, 57, 56, 30, 32, 58, 57, 32, 34, 59, 58, 34, 36, 60, 59, 36, 38, 61, 60, 38, 40, 62, 61, 40, 42, 63, 62, 44, 43, 64, 65, 43, 45, 66, 64, 45, 46, 67, 66, 46, 47, 68, 67, 47, 48, 69, 68, 48, 49, 70, 69, 49, 50, 71, 70, 50, 51, 72, 71, 51, 52, 73, 72, 52, 53, 74, 73, 53, 54, 75, 74, 54, 55, 76, 75, 55, 56, 77, 76, 56, 57, 78, 77, 57, 58, 79, 78, 58, 59, 80, 79, 59, 60, 81, 80, 60, 61, 82

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [14]:
vtk_grid("heat_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

1-element Vector{String}:
 "heat_equation.vtu"

In [27]:
using SparseArrays
using LinearAlgebra
using Arpack

# Create a sparse matrix (example)
n = 1000
A = spdiagm(0 => 2*ones(n), 1 => -ones(n-1), -1 => -ones(n-1))

# Solve the eigenproblem: find the smallest eigenvalues
nev = 1  # Number of eigenvalues to find
vals, vecs = eigs(A, nev; which=:SM, maxiter=10000)  # :SM for smallest magnitude eigenvalues

# Display the eigenvalues
println("Eigenvalues:")
println(vals)

# Optionally, display the corresponding eigenvectors
println("Eigenvectors (first eigenvector):")
println(vecs[:, 1])

Eigenvalues:
[9.849886676548702e-6, 3.939944968634821e-5, 8.864839796916608e-5, 0.00015759624642851196, 0.00024624231593604694, 0.0003545857333379025]
Eigenvectors (first eigenvector):
[0.00014028558300278356, 0.0002805697842084032, 0.00042085122183345517, 0.0005611285141216181, 0.0007014002793574955, 0.0008416651358801382, 0.0009819217020965, 0.0011221685964953674, 0.0012624044376607813, 0.0014026278442854352, 0.001542837435184735, 0.0016830318293102344, 0.001823209645762919, 0.0019633695038070577, 0.0021035100228842325, 0.0022436298226259097, 0.0023837275228681433, 0.0025238017436645162, 0.002663851105299582, 0.0028038742283032297, 0.0029438697334633313, 0.0030838362418402453, 0.003223772374779649, 0.0033636767539264303, 0.0035035480012383124, 0.003643384738999438, 0.003783185589833686, 0.003922949176718686, 0.004062674122998722, 0.004202359052399178, 0.004342002589039137, 0.004481603357445439, 0.004621159982566543, 0.004760671089785479, 0.004900135304933688, 0.005039551254304335, 0.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*